# <a id='toc1_'></a>[Bias-variance trade-off](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Bias-variance trade-off](#toc1_)    
  - [What is bias?](#toc1_1_)    
  - [What is variance?](#toc1_2_)    
  - [Extra: The maths of it all  ](#toc1_3_)    
    - [What is noise?](#toc1_3_1_)    
  - [What do want and how to get there?](#toc1_4_)    
    - [Regularization](#toc1_4_1_)    
      - [Train Test Split](#toc1_4_1_1_)    
    - [Baseline](#toc1_4_2_)    
    - [Revisit Train-Test split](#toc1_4_3_)    
    - [Review overfitting across multiple parameters](#toc1_4_4_)    
- [Resources](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

![](https://miro.medium.com/v2/resize:fit:828/format:webp/1*9hPX9pAO3jqLrzt0IE3JzA.png)  
(Source: [Understanding the Bias-Variance Tradeoff, Medium](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229))

## <a id='toc1_1_'></a>[What is bias?](#toc0_)

**Bias is how wrong our model is on average.** 

In maths terms, bias is the difference between our models predictions and the correct answer. A model has **high bias** if its predictions are very far off from the truth, typically because the model is too simple or the data seen by the model is not representative of the real data. 

If the issue is the model, we say that the model is **underfit** and even if we give the model more data, because it's so simple, it still can't learn. That's sad but don't worry, simple models are still good for a variety of tasks!

## <a id='toc1_2_'></a>[What is variance?](#toc0_)

**Variance is how different the predictions of our model are for the SAME type of data.** 

Imagine our house price predictor. If we train a model which has a high variance, it could tell us that a house with 3 rooms is $100,000 and one with 4 rooms is $500,000. Yikes! 

You notice the variance of a model when you look at an accuracy metric on 2 different sets, usually the train and test set (later on, train and validation...). 
* If the difference is high, the model has **high variance**, i.e. it memorized the data and didn't learn much, like a student who thinks they had the answer to the test but then sit the exam and get a bad score. The model is **overfit**.
* If the difference is low, the model has **low variance**, i.e. it learnt the main patterns in the data, like a student who studied the lessons and not the test answers! The model is neither overfit or underfit.

## <a id='toc1_3_'></a>[Extra: The maths of it all](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote12.html)   [&#8593;](#toc0_)

$\underbrace{E_{\mathbf{x}, y, D} \left[\left(h_{D}(\mathbf{x}) - y\right)^{2}\right]}_\mathrm{Expected\;Test\;Error} = \underbrace{E_{\mathbf{x}, D}\left[\left(h_{D}(\mathbf{x}) - \bar{h}(\mathbf{x})\right)^{2}\right]}_\mathrm{Variance} + \underbrace{E_{\mathbf{x}, y}\left[\left(\bar{y}(\mathbf{x}) - y\right)^{2}\right]}_\mathrm{Noise} + \underbrace{E_{\mathbf{x}}\left[\left(\bar{h}(\mathbf{x}) - \bar{y}(\mathbf{x})\right)^{2}\right]}_\mathrm{Bias^2}$

### <a id='toc1_3_1_'></a>[What is noise?](#toc0_)

It's the stuff you can't predict. As you already know, models are not perfect representations of reality. If we were able to perfectly model the universe, we wouldn't need Machine Learning and our noise or error would be 0. 

## <a id='toc1_4_'></a>[What do want and how to get there?](#toc0_)

A model complex enough to understand the data but not so complex that it memorizes the data.

![](http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png)  

(Source: [Understanding the Bias-Variance Tradeoff, Scott Fortmann-Roe](https://scott.fortmann-roe.com/docs/BiasVariance.html))

When spotting overfitting, we can deal with it in a number of ways:
- Reduce model complexity
    - Tweak hyperparameters, e.g. choose lower `max_depth`, `n_estimators`, etc.
    - Choose a simpler model, e.g. Linear/Logistic Regression
- Apply regulatization, i.e. L1 and L2 regularization
- Remove redundant features
- **Last resort**: Use more training data
    - readily available solution: move from a 80/20 split to a 85/15 or 90/10 split. Do NOT go beyond a 90/10 split.   
    **Caveat**: If you have a lot of data, e.g. >10,000 rows, you can start off with a 90/10 split or go beyond the 90/10 split.
    - more costly: gather more data

### <a id='toc1_4_1_'></a>[Regularization](#toc0_)

In [2]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
california = fetch_california_housing()
print(california["DESCR"])

In [ ]:
df_cali = pd.DataFrame(california["data"], columns = california["feature_names"])
df_cali["median_house_value"] = california["target"]

df_cali.head()

#### <a id='toc1_4_1_1_'></a>[Train Test Split](#toc0_)

In [5]:
features = df_cali.drop(columns = ["median_house_value","AveOccup", "Population", "AveBedrms"])
target = df_cali["median_house_value"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

### <a id='toc1_4_2_'></a>[Baseline](#toc0_)

In [ ]:
forest = RandomForestRegressor(n_estimators=20, max_depth=10, random_state=42)
forest.fit(X_train, y_train)
print("Train Score", forest.score(X_train, y_train))
print("Test Score", forest.score(X_test, y_test))

### <a id='toc1_4_3_'></a>[Revisit Train-Test split](#toc0_)

In [ ]:
df_cali.shape

In [ ]:
20640 * 0.1 # 2K samples to test on

In [35]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.9, random_state=0)

In [ ]:
forest = RandomForestRegressor(n_estimators=20, max_depth=10, random_state=42)
forest.fit(X_train, y_train)
print("Train Score", forest.score(X_train, y_train))
print("Test Score", forest.score(X_test, y_test))

### <a id='toc1_4_4_'></a>[Review overfitting across multiple parameters](#toc0_)

In [ ]:
results = pd.DataFrame()

for n_estimators in range(10, 100, 10):
    result = {}
    forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=10, random_state=42)
    forest.fit(X_train, y_train)

    result["Score"] = forest.score(X_train, y_train)
    result["Set"] = "Train Set"
    result["Parameter"] = n_estimators
    results = pd.concat([results, pd.DataFrame([result])], axis=0)

    result["Score"] = forest.score(X_test, y_test)
    result["Set"] = "Test Set"
    results = pd.concat([results, pd.DataFrame([result])], axis=0)

In [ ]:
px.line(results, y="Score", x="Parameter", color="Set")

In [28]:
results = pd.DataFrame()

for max_depth in range(1, 20, 2):
    result = {}
    forest = RandomForestRegressor(n_estimators=20, max_depth=max_depth, random_state=42)
    forest.fit(X_train, y_train)

    result["Score"] = forest.score(X_train, y_train)
    result["Set"] = "Train Set"
    result["Parameter"] = max_depth
    results = pd.concat([results, pd.DataFrame([result])], axis=0)

    result["Score"] = forest.score(X_test, y_test)
    result["Set"] = "Test Set"
    results = pd.concat([results, pd.DataFrame([result])], axis=0)

In [ ]:
px.line(results, y="Score", x="Parameter", color="Set")

In [39]:
results = pd.DataFrame()

for min_samples_split in range(10, 100, 10):
    result = {}
    forest = RandomForestRegressor(n_estimators=20, max_depth=7, random_state=42, min_samples_split=min_samples_split)
    forest.fit(X_train, y_train)

    result["Score"] = forest.score(X_train, y_train)
    result["Set"] = "Train Set"
    result["Parameter"] = min_samples_split
    results = pd.concat([results, pd.DataFrame([result])], axis=0)

    result["Score"] = forest.score(X_test, y_test)
    result["Set"] = "Test Set"
    results = pd.concat([results, pd.DataFrame([result])], axis=0)

In [ ]:
px.line(results, y="Score", x="Parameter", color="Set")

# <a id='toc2_'></a>[Resources](#toc0_)

* StatQuest:
    * [Bias & Variance](https://www.youtube.com/watch?v=EuBBz3bI-aA) - 6 min